In [ ]:
import numpy as np
import scipy.io
mat = scipy.io.loadmat('dataset_connected_NYC.mat') 

In [ ]:
friendship_new = mat['friendship_new']
friendship_old = mat['friendship_old']#new_friendship_old
selected_checkins = mat['selected_checkins'] #new_checkins
selected_users_IDs = mat['selected_users_IDs'] #new_users_IDs

In [ ]:
# 1. rebuild node index
offset1 = max(selected_checkins[:,0]);
dumy, dumy, n = np.unique(selected_checkins[:,1],return_index=True,return_inverse=True, axis=0)


selected_checkins[:,1] = n+1#+offset1; #n is the indices of the unique values in selected_checkins[:,1]
offset2 = max(selected_checkins[:,1]);
dumy, dumy, n = np.unique(selected_checkins[:,2],return_index=True,return_inverse=True, axis=0)

selected_checkins[:,2] = n+1#+offset2;
offset3 = max(selected_checkins[:,2]);
dumy, dumy, n = np.unique(selected_checkins[:,3],return_index=True,return_inverse=True, axis=0)
selected_checkins[:,3] = n+1#+offset3;

num_node_total = max(map(max, selected_checkins)) #max of the entire matrix = 8117

In [ ]:
#verify sequence
selected_checkins-=1
print(selected_checkins.shape)
print(min(selected_checkins[:,0]))
print(max(selected_checkins[:,0]))
print(min(selected_checkins[:,1]))
print(max(selected_checkins[:,1]))
print(min(selected_checkins[:,2]))
print(max(selected_checkins[:,2]))
print(min(selected_checkins[:,3]))
print(max(selected_checkins[:,3]))


In [ ]:
num_of_users = max(selected_checkins[:,0])
num_of_time = max(selected_checkins[:,1])
num_of_venues = max(selected_checkins[:,2])
num_of_categories = max(selected_checkins[:,3])

In [ ]:
print(num_of_users)
print(num_of_time)
print(num_of_venues)
print(num_of_categories)

In [ ]:
#separating training and testing data
train_size = int((80/100)*np.size(selected_checkins,0))
train_data = selected_checkins[:train_size]
nums_type = np.array([num_of_users+1, num_of_time+1, num_of_venues+1, num_of_categories+1])
test_data = selected_checkins[train_size:]
#data = {'train_data':train_data,'test_data':test_data,'num_types':num_types}

In [ ]:
np.savez_compressed('lbsndata/train_data.npz', train_data = train_data, nums_type = nums_type)
np.savez_compressed('lbsndata/test_data.npz', test_data = test_data, nums_type = nums_type)


In [ ]:
print(train_data.shape)

In [ ]:
#selected_checkins format -
#selected_checkins (4 columns): user_index, time (hour in a week), venue_index, venue_category_index
#feeding it as it is to DHNE

#run hypergraph_embedding.py --data_path lbsndata --save_path lbsnresult -s '10 10 10 10' -e 2
!bash run_dhne.sh

In [ ]:
#embeddings are stored in lbsnresult/model_(embedding_size)_(hidden_size)_(epochs)_(batch_size)_(learn-
# ing_rate)_(neg_sam)/embeddings.npy
#corresponding model stored in lbsnresult/model_(no.ofemneddings)/model.h5

embed = np.load('lbsnresult/model_100/embeddings.npy')
print(embed.shape)
print('embed[0].shape',embed[0].shape)
print('embed[1].shape',embed[1].shape)
print('embed[2].shape',embed[2].shape)
print('embed[3].shape',embed[3].shape)

In [ ]:
#location prediction on the learnt embeddings
#selected_checkins (4 columns): user_index, time (hour in a week), venue_index, venue_category_index
import time as tm
from scipy.spatial import distance

count = 0

start = tm.time()
for d in range(test_data.shape[0]):
    test = test_data[d]
    user = test[0]
    time = test[1]
    dist = []
    
    for i in range(min(test_data[:,2]),max(test_data[:,2])+1):
        summ = (1 - distance.cosine(embed[0][user],embed[2][i]))+(1 - distance.cosine(embed[1][time],embed[2][i]))
        dist.append((abs(summ),i))
        
    dist = sorted(dist, key=lambda student: student[0])
    if test[2] in (np.array(dist[0:20])[:,1]):
        count+=1
        
ending = tm.time()

print("this is count ",count)
print("this is count fraction ",count/test_data.shape[0])
print("time lapsed ",ending-start)